# Previsão da Progressão dos Casos Confirmados de COVID19 por Estado no Brasil

### 

# Importando Módulos

In [1]:
%%time
#Data Wrangling
from datetime import datetime as dt
import pandas as pd
import numpy as np

#Data Transform
from category_encoders import OneHotEncoder

#Web Scraping
from tabulate import tabulate
import urllib.request
from bs4 import BeautifulSoup
import requests

#Machine Learning - Algorithms
from sklearn.linear_model import Ridge

#Machine Learning - Save
from sklearn.externals import joblib

Wall time: 3.56 s


C:\Users\KÁTIA\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Data Processing

In [2]:
%%time
#Carregando Dados
df = pd.read_csv("C:/Users/KÁTIA/Desktop/EVOLUTION 2020/COVID/UPDATE/brazil_covid19.csv",sep=',')

#Removendo coluna 'region'
df=df.drop(columns='region')

#Convertendo 'date' de string para datetime
df.date=pd.to_datetime(df.date)

#Ordenando dartagrame por estado e data
df=df.sort_values(by=['state','date'])

df

Wall time: 82 ms


## Incluindo a UF e região de cada estado

In [3]:
%%time
#Retirando dados da web
res=requests.get('https://www.estadosecapitaisdobrasil.com/')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]

#Criando dataframe
uf_reg = pd.read_html(str(table))
uf_reg=pd.DataFrame(uf_reg)
uf_reg=uf_reg[0][0]
uf_reg.head(2)

Wall time: 816 ms


In [4]:
%%time
#Renomeando e definindo com índice a coluna chave do dataframe
uf_reg=uf_reg.rename(columns={'Estado':'state','Sigla':'UF'})
uf_reg=uf_reg.set_index(uf_reg.state)

#Juntando os datasets
df=df.merge(uf_reg[['Região','UF']],how="left",on='state')
df

Wall time: 9 ms


## Criando variável 'Day'
### Para contabilizar o número de dias após o primeiro caso suspeito

In [5]:
%%time
#Datas únicas de forma ordenada
df.date.unique()

#Criando dataframe
start=pd.DataFrame(np.arange(1,len(df.date.unique())+1),index=df.date.unique(),columns=['Day'])
start.index.name='date'

#Juntando os dataframes
df=df.merge(start['Day'], how='left', on='date')
df

Wall time: 57 ms


# 

# Save and Load Data

In [6]:
%%time
#Salvando os dados
df.to_csv('teste_produto.csv',sep=';',index=False)

#Carregando o dataset
df=pd.read_csv('teste_produto.csv',sep=';')
#Convertendo 'date' de string para datetime
df.date=pd.to_datetime(df.date)
df.tail(5)

Wall time: 290 ms


# 

# Feature Engineering

In [7]:
%%time
#Copiando dataframe original
date_state=df.copy()

#Relacionadas ao dia anterior (d-1)
#Número de Casos Confirmados por Estado
date_state['Cases_Last_Day']=date_state.groupby(['state']).cases.shift()

#Criando novas colunas para transformação
date_state['UF_CAT']=date_state.UF

#Transformando as variáveis
ohe_uf = OneHotEncoder(cols=['UF_CAT'],use_cat_names=True,drop_invariant=True)
date_state = ohe_uf.fit_transform(date_state)

date_state

Wall time: 62 ms


# 

# Save and Load Data

In [8]:
%%time
#Salvando os dados
date_state.to_csv('teste_produto.csv',sep=';',index=False)

#Carregando os dados
date_state=pd.read_csv('teste_produto.csv',sep=';')
#Convertendo date de string para datetime
date_state.date = pd.to_datetime(date_state.date)
#Substituindo dados nulos por 0
date_state=date_state.fillna(0)

#Verificando o shape do dataset
date_state.shape

Wall time: 125 ms


# 

# Modelagem
## Data Validation

In [9]:
%%time
#Último dia
date_state.Day.max()

Wall time: 1 ms


80

In [10]:
%%time
#Divisão dos dados
#Dias de início e fim, respecivamente
mn, mx = date_state.Day.max()-31 , date_state.Day.max()

#Visualizando a proporção inicial de treino e validação
date_state[date_state['Day']<mn].shape
date_state[date_state['Day']>=mn].shape[0]
print("Dados de Validação: {:.2f}%".format(date_state[date_state['Day']>=mn].shape[0]/(date_state.shape[0])*100))

Dados de Validação: 40.00%
Wall time: 10 ms


# Métricas

In [11]:
%%time
#Criando métricas utilizando funções

#Root Mean Squared Logarithmic Error
def RMSLE(pred,target):
    
    pred=np.array(pred)
    target=np.array(target)
    
    return np.sqrt(np.mean((np.log1p(abs(pred))-np.log1p(abs(target)))**2))

Wall time: 0 ns


# 

# Modelagem

In [12]:
%%time
#Visualizando variáveis finais do dataset
date_state.columns.values

Wall time: 0 ns


array(['date', 'state', 'cases', 'deaths', 'Região', 'UF', 'Day',
       'Cases_Last_Day', 'UF_CAT_AC', 'UF_CAT_AL', 'UF_CAT_AP',
       'UF_CAT_AM', 'UF_CAT_BA', 'UF_CAT_CE', 'UF_CAT_DF', 'UF_CAT_ES',
       'UF_CAT_GO', 'UF_CAT_MA', 'UF_CAT_MT', 'UF_CAT_MS', 'UF_CAT_MG',
       'UF_CAT_PR', 'UF_CAT_PB', 'UF_CAT_PA', 'UF_CAT_PE', 'UF_CAT_PI',
       'UF_CAT_RN', 'UF_CAT_RS', 'UF_CAT_RJ', 'UF_CAT_RO', 'UF_CAT_RR',
       'UF_CAT_SC', 'UF_CAT_SE', 'UF_CAT_SP', 'UF_CAT_TO'], dtype=object)

In [13]:
%%time
#Selecionando as features necessárias
features = ['Cases_Last_Day', 'UF_CAT_TO', 'UF_CAT_SE', 'UF_CAT_AL', 'UF_CAT_MS', 'UF_CAT_GO', 'UF_CAT_AC']
#Quantidade selecionada
len(features)

Wall time: 0 ns


## Ridge Regressor

In [14]:
%%time
models=[Ridge(random_state=0,alpha=0.01,normalize=True,fit_intercept=True)]

name_mod=['RID']

feat_mod = [features]

for num in range(0,len(models)):
    
    error2=[]
    
    for i in range(mn,mx+1):
        if date_state[date_state['Day']==i].shape[0] !=0:

            train=date_state[date_state['Day']<i]
            test=date_state[date_state['Day']==i]

            sel=feat_mod[num]
            X_train=train.loc[:,sel].values
            y_train=train['cases'].values
            X_test=test.loc[:,sel].values
            y_test=test['cases'].values

            model=models[num]
            model.fit(X_train,y_train)

            y_pred=model.predict(X_test)
                
            e2 = RMSLE(y_pred,y_test)
            error2.append(e2)              

    #Imprimir mensagem para cada algorimo mostrando os scores
    print("\n{}\nMédia do Erro RMSLE: {:.4f}".format(name_mod[num],np.mean(error2)))
    
    #Salvando o modelo
    joblib.dump(model,'model_{}.pkl'.format(name_mod[num]))


RID
Média do Erro RMSLE: 0.1517
Wall time: 930 ms


## Load Model

In [15]:
%%time
#Carregando e visualizando os parâmetros do modelo
model=joblib.load('model_RID.pkl')
print(model,'\n\n',model.coef_)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=0, solver='auto', tol=0.001) 

 [ 1.07675726 -5.70592708 -5.77462128 -5.12253634 -5.58068256 -4.68725024
 -5.29508261]
Wall time: 12 ms
